In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


ModuleNotFoundError: No module named 'sklearn'

# Load the data

When downloading our data, it did not come with headers. We fixed this by adding getting the ordered headers from directory we downloaded the data frame, and implemented then BY USING... From this group of files, we confirmed our theories that variable_name_SE was the standard error and variable_name_Worst was the largest value. Along with the leading diagonal of our pair plots of all the unique variables following a normal distributions, inline with the Central Limit Theorem, gave us confidence we have assigned the correct headers onto our dataframe.

Originally our data was in .name and .data file formats, which we could not access. We fixed this by converting the files into a .csv format. This gave us an error that we might lose information, however according to the data set summary we still have every instance of our data and every column, and it is to the same 4 significant figures-so we know no data has been lost.


In [ ]:
df = pd.read_csv("./wdbc.csv")
df.head(10)

### Clean the data

Our data is supposedly clean, and observing the first few instances it does seem so. We already know that Pandas only allows one data type per column, so this is a non issue. We will look for duplicates in our data, as the probability of so many factors being identical is significantly low- if we have duplicates this may be a fault in the data collection or the previously mentioned .csv method of extracting the data. One issue that we cannot determine just from looking at the data, is the data type of our values, i.e a string for a float.

In [ ]:
print(f"Same patient entered more than once: {df.duplicated().any()}")
df_dup = df.drop('ID', axis=1)
print(f"Same patient entered more than once under a different ID: {df_dup.duplicated().any()}") # testing with ID
print(f"Null values? {df.isnull().values.any()}")
print(f"There are {len(df)} instances, of which {df['ID'].nunique()} are unique \n")
df.info()


We will next split our data in its label and features, and drop our ID column- as it bears no effect on our model (and as each instance value overall is unique, we could always reverse engineer our way back to get the ID). As seen above, our B/M column has an object type - this can cause issues with Seaborn plot so we will encode it 0 or 1,

In [ ]:
encoder = LabelEncoder().fit(df["B/M"])
df['B/M'] = encoder.transform(df["B/M"])
label = df["B/M"] #One line here is redundant?
not_data = ["ID", "B/M"]
feature = df.drop(not_data, axis=1)
data = pd.concat([label, feature], axis=1)

We can also preemptively split our data into the train/test/validation sets, however these ratios be tuned at a later date depending on the number of features in our data and the specific algorithm. We used a stratified sample, as this is a classification problem with our labels being either Benign(0) or Malignant(1). We do not have to shuffle our dataframe, as train_test_split(shuffle=True).


In [ ]:
x, x_test, y, y_test = train_test_split(feature, label, test_size=0.25, stratify=label)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.25, stratify=y)

### Data Visualization & Exploration

In [ ]:
sns.countplot(data=feature, x=label)
plt.title("Diagnosis")
plt.show()
# Make graphs nice
sum(label) # 1 mean malignant
len(label)
print(f"Percentage of Malignant Tumours is {round(sum(label) / len(label), 3)}")

The ratio of Benign is slightly higher than what we would expect (as ~80% of breast cancer on average is Benign), so our data is not too imbalanced. However, we may still decide to try penalized models as a) percentage of Malignant tumours is higher than normal (so we would penalize malignant) and b) the cost of false negatives is very high (as treatment will be delayed) so we may want to significantly penalize false negatives in our model.

In [ ]:
sns.pairplot(data=feature.iloc[::, :10])
plt.show()
# can't load for me, also how can I add hue to this ('B/M')

In [ ]:
f, ax = plt.subplots(figsize=(18, 18))

sns.heatmap((data.iloc[::, :10]).corr(), annot=True, fmt=".1f", ax=ax)
plt.title("Feature Correlation")
plt.show()

Look at the correlations between B/M and other varaibles, i.e symmetry is only 0.3 so should we remove this vairable?